her bir bilet için BUSINESS, LEISURE, SECOND HOME ve STUDENT yolculuk nedenlerini tahmin etmek
Oluşan sonuçların kullanım senaryolarına ilişkin açıklama yapmanız beklenmektedir. 

eda for numerical values
distribution for all variables
is there any outlier
any duplicate

eda- histograms- visualizations - boxplots 
look at feature importance and remove the unimportants


classification
class weights
different classifiers
tuning

https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/pages/introduction.html
https://shap.readthedocs.io/en/latest/index.html

https://docs.rapids.ai/api/cudf/stable/        
https://www.kaggle.com/rohanrao/tutorial-on-reading-large-datasets?scriptVersionId=49328159&cellId=14

https://towardsdatascience.com/wisdom-of-the-crowd-voting-classifier-bagging-pasting-random-forest-and-extra-trees-289ef991e723

    

In [1]:
from pandas_profiling import ProfileReport
import pandas as pd

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
merged_df = pd.read_csv('merged.csv', index_col='Unnamed: 0')
merged_df.shape

(1295571, 47)

In [4]:
label =  ['FLIGHT_REASON']

categorical_str = [
'ARR',
'DEP',
'ID_PNR_ITI_CABCLASS_1',
'ID_PNR_ITI_CABCLASS_2',
'ID_PNR_ITI_SELCLASS_1',
'ID_PNR_ITI_SELCLASS_2',
'JRNY_TYP',
'NTNLT1' ] 

categorical_int = [
'arrival_interval_1',
'arrival_interval_2',
'flight_month_1',
'flight_month_2',
'flight_weekday_1',
'flight_weekday_2',
'pnr_create_month',
'pnr_create_weekday',
'pnr_workhour_interval']

numerical = [
'CUST_AGE',
'diff_pnr_firstflight', 
'diff_second_firstflight',
'flight_duration_1',
'flight_duration_2',
]

passthrough = [
'CHILD_FLG',
'FAMILY_FLG',
'INFANT_FLG',
'PET_FLG',
'POS_POC_SAME_FLG',
'SAME_SRNAME_FLG',
'SEAT_SELECT_FLG',
'SPORT_FLG',
'XBAG_FIRST_FLT_FLG',
'XBAG_FLG',
'XBAG_LAST_FLT_FLG',
'XBAG_TWO_WAY_FLT_FLG',
'PNR_PSSG_COUNT',
'num_of_flights']

drops = [
'ID_PNR_CREATION_DTIME_GMT',
'ID_PNR_CREATION_YMD',
'ID_PNR_ITI_ARR_APT_1',
'ID_PNR_ITI_ARR_APT_2',
'ID_PNR_ITI_DEP_APT_1',
'ID_PNR_ITI_DEP_APT_2',
'ID_TKT_NO',
'OND',
'PNR_NO',
'date']

In [5]:
merged_df = merged_df[categorical_str+categorical_int+numerical+passthrough+label]

In [6]:
len(merged_df.columns)

37

### drop duplicates

In [7]:
tmp_df = merged_df.copy()
tmp_df['values_nolabel']  = merged_df.drop(['FLIGHT_REASON'], axis=1).applymap(str).apply(' '.join, axis=1)
tmp_df['values_label']  = merged_df.applymap(str).apply(' '.join, axis=1)

In [8]:
no_label = tmp_df[tmp_df.duplicated(subset=['values_nolabel'])].index
with_label = tmp_df[tmp_df.duplicated(subset=['values_label'])].index

In [9]:
conflicts_indexes = merged_df.loc[list(set(no_label).difference(set(with_label)))].index
conflicts_indexes

Int64Index([1201984, 566526, 1110212, 1226264, 807898, 1247613, 1260318], dtype='int64')

In [10]:
all_conflict_indexes = []
for ind in conflicts_indexes:
    print(tmp_df[tmp_df['values_nolabel'] == tmp_df.loc[ind, 'values_nolabel']]['FLIGHT_REASON'])
    all_conflict_indexes.extend(list(tmp_df[tmp_df['values_nolabel'] == tmp_df.loc[ind, 'values_nolabel']].index))
    print('---')

898909      LEISURE
1201984    BUSINESS
Name: FLIGHT_REASON, dtype: object
---
343385     LEISURE
566526    BUSINESS
Name: FLIGHT_REASON, dtype: object
---
343581     BUSINESS
343703     BUSINESS
1110212     LEISURE
Name: FLIGHT_REASON, dtype: object
---
980116      LEISURE
1226264    BUSINESS
Name: FLIGHT_REASON, dtype: object
---
638445     LEISURE
807898    BUSINESS
Name: FLIGHT_REASON, dtype: object
---
357281         LEISURE
357282         LEISURE
1247613    SECOND HOME
Name: FLIGHT_REASON, dtype: object
---
868592      LEISURE
1260318    BUSINESS
Name: FLIGHT_REASON, dtype: object
---


In [11]:
merged_df.drop(conflicts_indexes, inplace=True)

In [12]:
merged_df.drop_duplicates(inplace=True)

In [13]:
merged_df.shape

(953652, 37)

In [14]:
import seaborn as sns
import matplotlib.pyplot as plt

### correlation

In [15]:
matrix = merged_df.corr().abs()

# Create a mask
mask = np.triu(np.ones_like(matrix, dtype=bool))

# Create a custom diverging palette
cmap = sns.diverging_palette(250, 15, s=75, l=40, n=9, center="light", as_cmap=True)
plt.figure(figsize=(16, 12))
sns.heatmap(matrix[matrix>0.5], mask=mask, center=0, annot=True, fmt='.2f', square=True, cmap='crest')

plt.show();

NameError: name 'np' is not defined

In [ ]:
# dropping because high correlation
merged_df.drop(['XBAG_FLG'], axis=1, inplace=True)

In [ ]:
matrix = merged_df.corr().abs()

# Create a mask
mask = np.triu(np.ones_like(matrix, dtype=bool))

# Create a custom diverging palette
cmap = sns.diverging_palette(250, 15, s=75, l=40, n=9, center="light", as_cmap=True)
plt.figure(figsize=(16, 12))
sns.heatmap(matrix[matrix>0.5], mask=mask, center=0, annot=True, fmt='.2f', square=True, cmap='crest')

plt.show();

In [ ]:
merged_df.columns

In [ ]:
sns.pairplot(merged_df, hue='FLIGHT_REASON', height=3, diag_kind = 'hist')

In [ ]:
sns.pairplot(merged_df, diag_kind = 'hist')

In [ ]:
sns.histplot(data=merged_df, x="DEP", )

In [200]:
profile = ProfileReport(merged_df, title="Pandas Profiling Report", explorative=True)
profile

Summarize dataset:   0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
y = merged_df['FLIGHT_REASON']
X = merged_df[categorical_str+categorical_int+numerical+passthrough]

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X.isna().sum()[X.isna().sum()>0]

In [ ]:
X[numerical].info()

In [ ]:
merged_df['ID_PNR_ITI_SELCLASS_2'].unique() 

In [ ]:
(X[numerical]==-1).index

In [ ]:
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [ ]:
cw = dict(zip(np.unique(y), compute_class_weight('balanced', np.unique(y), y)))

In [ ]:
c

In [ ]:
%%time
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_str_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='None')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))])

categorical_int_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-99)),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
                                transformers=[
                                              ('num', numeric_transformer, numerical),
                                              ('cat_str', categorical_str_transformer, categorical_str),
                                              ('cat_int', categorical_int_transformer, categorical_int),], 
                                remainder='passthrough')

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(class_weight=))])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)

clf.fit(X_train, y_train)
print("training score: %.3f" % clf.score(X_train, y_train))
print("test score: %.3f" % clf.score(X_test, y_test))

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
stopp

In [ ]:
without class weights:
              precision    recall  f1-score   support

    BUSINESS       0.97      0.99      0.98    151336
     LEISURE       0.98      0.97      0.97    190324
 SECOND HOME       0.98      0.97      0.98     85474
     STUDENT       0.99      0.42      0.59       405

    accuracy                           0.98    427539
   macro avg       0.98      0.84      0.88    427539
weighted avg       0.98      0.98      0.98    427539

In [ ]:
merged_df.diff_pnr_firstflight

In [ ]:
new features
if we can use additional datasets
is NTNLT1 in arr or dep of oene of the flights 

In [ ]:
is it possible to flight before pnr

pnrs[pnrs['diff_pnr_firstflight']<0][['ID_PNR_ITI_DEP_YMD_1', 'ID_PNR_CREATION_DTIME_GMT', 'diff_pnr_firstflight']]

2. uçuşun 1. uçuştan önce olması mümkün mü?
pnrs[pnrs['diff_second_firstflight']==2][['ID_PNR_ITI_DEP_YMD_2', 'ID_PNR_ITI_DEP_YMD_1', 'diff_second_firstflight']]
